In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [2]:
import pandas as pd
print(pd.__version__)

1.0.5


In [3]:
import seaborn as sns

Informacion de las columnas
- item_id
- description: Descripcion del shipping (por ejemplo, free shipping, letter, priority mail, etc)
- est_delivery: Tiempo estimado de entrega (generalmente en días)
- price: Precio de la opción de envio [BTC]
- timestamp: Fecha y hora en la que se scrapeo

In [ ]:
shipping_skilroad = pd.read_csv('csv/silkroad_shippings.csv')
shipping_skilroad.head()

In [ ]:
bitcoin = pd.read_csv('csv/bitcoins.csv')
bitcoin

In [ ]:
shipping_skilroad.rename(columns={'est_delivery':'delivery_days'},inplace=True)
shipping_skilroad.head()

In [ ]:
shipping_skilroad['timestamp'] = pd.to_datetime(shipping_skilroad['timestamp'])
bitcoin['Date'] = pd.to_datetime(bitcoin['Date'])

In [ ]:
def make_numeric(x):
    num = ""
    for letter in x:
        if letter.isnumeric():
            num +=letter
    return num

In [ ]:
shipping_skilroad['delivery_days'] = shipping_skilroad['delivery_days'].map(make_numeric)
shipping_skilroad['delivery_days']

In [ ]:
shipping_skilroad['date'] =shipping_skilroad['timestamp'].dt.date

In [ ]:
bitcoin.rename(columns={'Date':'date'},inplace=True)
bitcoin

In [ ]:
bitcoin.date = bitcoin.date.astype('datetime64[ns]')
shipping_skilroad.date = shipping_skilroad.date.astype('datetime64[ns]')

In [ ]:
merged = shipping_skilroad.merge(bitcoin,on='date',how='left')
shipping = merged[['item_id','description','delivery_days','price','Price','date']]

In [ ]:
shipping['EU_price'] = shipping['Price'] * shipping['price']

In [ ]:
# Filtro, aquellos shippings que poseen precios irreales, dejando como maximo 5000
real_price_shipping = shipping[(shipping['EU_price'] >= 0) & (shipping['EU_price'] <= 5000)]
real_price_shipping

In [ ]:
# Limpio la memoria
del shipping

In [4]:
shippin_no_id = real_price_shipping[['description','delivery_days','EU_price','date']]
shippin_no_id

NameError: name 'real_price_shipping' is not defined

In [ ]:
def a(x):
    if x == '':
        return 0
    else:
        return x

In [ ]:
shippin_no_id['delivery_days'] = shippin_no_id['delivery_days'].map(a)
shippin_no_id.info()

In [ ]:
shippin_no_id['delivery_days'].fillna(0,inplace=True)

In [ ]:
shippin_no_id['delivery_days'].value_counts()

In [ ]:
shippin_no_id['delivery_days'] = shippin_no_id['delivery_days'].astype(int)

In [ ]:
# Para la descripcion, quizas sea conveniente usar un hot-encoding, me fijo cuantas valores unicos posee description
shippin_no_id['description'].nunique()

In [ ]:
shippin_no_id['description'].fillna('unknown',inplace=True)

In [ ]:
# Como hay 4040 muestras unicas, un one-hot-encoding será problematico.
# Por lo que usaré un label encoding para description
shippin_no_id.info()

## Modelado

Antes de armar los encondigs, separo mi dataset en validation y training.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error

In [ ]:
y = shippin_no_id.EU_price
X = shippin_no_id.drop(['EU_price'],axis=1)

In [ ]:
# Para lograr un mejor encoding, separare las fechas en columnas distintas
X['year'] = X['date'].dt.year
X['month']= X['date'].dt.month
X['day']= X['date'].dt.day
X

In [ ]:
del X['date']

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
# Hago un label enconding para description
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
X_train['description'] = ordinal_encoder.fit_transform(X_train['description'].astype(str).values.reshape(-1,1))
X_train

In [ ]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train);

In [ ]:
# Hago los encodings del validation
X_validation['description'] = ordinal_encoder.transform(X_validation['description'].astype(str).values.reshape(-1,1))
X_validation

In [ ]:
predictions = rf.predict(X_validation)

In [ ]:
predictions = pd.Series(predictions)
predictions

In [ ]:
y_validation = y_validation.reset_index()

In [ ]:
del y_validation['index']

In [ ]:
y_validation.loc[y_validation['EU_price'] == 'NaN']

In [ ]:
mean_absolute_error(predictions,y_validation['EU_price'])

In [ ]:
errors.shape

In [ ]:
predictions

In [ ]:
result = pd.concat([y_validation,predictions],axis=1)
result

In [ ]:
result.rename(columns={0:'EU_price_prediction'},inplace=True)
result

In [ ]:
result['relative_error'] = abs((result['EU_price'] -result['EU_price_prediction'])/result['EU_price'])
result

In [ ]:
sns.set(rc={"figure.dpi":100,'figure.figsize':(6.4,6.4)})
ax= sns.regplot(data=result, x="EU_price", y="EU_price_prediction",color="g")
ax.set_title("Diagrama de dispersión entre el shipping real y el predicho",fontsize=10)
ax.set_xlabel("real",fontsize=8)
ax.set_ylabel("prediccion",fontsize=8)

In [ ]:
sns.set_theme(style="whitegrid")
sns.lineplot(data=result,x='EU_price',y='relative_error')